In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer
# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    dtype="auto",
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from transformers import pipeline

# Create a pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
    max_new_tokens=200, # Reduced max_new_tokens
    do_sample=False,
    use_cache=False # Disable caching
)

Device set to use cuda
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [3]:
# @title
# The promt (user input / query)
messages = [
    {"role": "user", "content": "Create a summary for a Youtube video suggestion on how to build an LLM project."}
]

# Generate output
output = generator(messages)
print(output[0]["generated_text"])

 Title: Building Your First Large Language Model (LLM) Project

Summary:

In this video, we will guide you through the process of building your first Large Language Model (LLM) project. We will cover the essential steps, from understanding the basics of LLMs to implementing them using popular frameworks and tools.

The video will start with an introduction to LLMs, their applications, and the benefits they offer. We will then dive into the technical aspects of building an LLM, including data collection, preprocessing, and model training.

We will also discuss the different types of LLMs, such as transformer-based models, and the various frameworks and tools available for building them, such as TensorFlow, PyTorch, and Hugging Face.

Throughout the video, we will provide practical examples and code snippets to help you understand the concepts and implement them in your project. We


In [13]:
# Tokens and embeddings
prompt = "Write an email to my subscribers how to achieve their goals by constantly upskilling and believing in themselves. <|assistant|>"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# Generate the text
generation_output = model.generate(
    input_ids=input_ids,
    max_new_tokens=20,
    use_cache=False # Disable caching to fix the AttributeError
)

# Print the output
print(tokenizer.decode(generation_output[0]))

Write an email to my subscribers how to achieve their goals by constantly upskilling and believing in themselves. <|assistant|> Subject: Unlock Your Potential: Achieve Your Goals Through Upskilling and


In [14]:
input_ids

tensor([[14350,   385,  4876,   304,   590, 21696,  2596,   920,   304,  6176,
          1009, 14433,   491, 21003, 24081, 29895,  8873,   322,  1339, 15387,
           297,  6053, 29889, 29871, 32001]], device='cuda:0')

In [20]:
for id in input_ids[0]:
  print(tokenizer.decode(id))

Write
an
email
to
my
subscri
bers
how
to
achieve
their
goals
by
constantly
ups
k
illing
and
bel
ieving
in
themselves
.

<|assistant|>


In [17]:
generation_output

tensor([[14350,   385,  4876,   304,   590, 21696,  2596,   920,   304,  6176,
          1009, 14433,   491, 21003, 24081, 29895,  8873,   322,  1339, 15387,
           297,  6053, 29889, 29871, 32001,  3323,   622, 29901,   853,   908,
          3575, 10173,  2556, 29901, 15542,  2418,  3575,  2921,  1338, 17044,
           501,   567, 29895,  8873,   322]], device='cuda:0')

In [23]:
for output in generation_output[0]:
  print(tokenizer.decode(output))

Write
an
email
to
my
subscri
bers
how
to
achieve
their
goals
by
constantly
ups
k
illing
and
bel
ieving
in
themselves
.

<|assistant|>
Sub
ject
:
Un
lock
Your
Pot
ential
:
Ach
ieve
Your
Go
als
Through
U
ps
k
illing
and


In [24]:
generation_output

tensor([[14350,   385,  4876,   304,   590, 21696,  2596,   920,   304,  6176,
          1009, 14433,   491, 21003, 24081, 29895,  8873,   322,  1339, 15387,
           297,  6053, 29889, 29871, 32001,  3323,   622, 29901,   853,   908,
          3575, 10173,  2556, 29901, 15542,  2418,  3575,  2921,  1338, 17044,
           501,   567, 29895,  8873,   322]], device='cuda:0')

In [26]:
print(tokenizer.decode(501))
print(tokenizer.decode(567))
print(tokenizer.decode(29895))
print(tokenizer.decode(8873))
print(tokenizer.decode([501,567,29895,8873]))

U
ps
k
illing
Upskilling


The tokenization methods are chosen by the model creators. The popular methods are

*   byte pair encoding  (BPE) -- widely used by GPT models
*   WordPiece -- used by BERT (bidirectional encoder representations from transformers)

In [34]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""

In [35]:
# Comparing trained LLM tokenizers

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )

In [36]:
show_tokens(text, "bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " three tab ##s : " " 12 . 0 * 50 = 600 [SEP] 

In [38]:
show_tokens(text, "bert-base-cased")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " Three ta ##bs : " " 12 . 0 * 50 = 600 [SEP] 

In [40]:
show_tokens(text, "gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]


 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"        "  Three  tabs :  "              " 
 12 . 0 * 50 = 600 
 

In [41]:
input_ids

tensor([[14350,   385,  4876,   304,   590, 21696,  2596,   920,   304,  6176,
          1009, 14433,   491, 21003, 24081, 29895,  8873,   322,  1339, 15387,
           297,  6053, 29889, 29871, 32001]], device='cuda:0')

gpt2 differs from the tokenization shown in book, possibly updated

In [44]:
show_tokens(text, "google/flan-t5-small")

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

English and CA PI TAL IZ ATION  <unk>  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " Three tab s : " " 12. 0 * 50 = 600  </s> 

In [46]:
show_tokens(text, "Xenova/gpt-4")

tokenizer_config.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]


 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         "
 12 . 0 * 50 = 600 
 

In [47]:
show_tokens(text, "bigcode/starcoder2-15b")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]


 English  and  CAPITAL IZATION 
 � � �   � � 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

for the starcoder2 no asccess request needed

In [48]:
show_tokens(text, "facebook/galactica-1.3b")

tokenizer_config.json:   0%|          | 0.00/166 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.00 [00:00<?, ?B/s]


 English  and  CAP ITAL IZATION 
 � � � �  � � � 
 show _ tokens  False  None  elif   ==   > =  else :  two  t abs : "      "  Three  t abs :   "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

In [49]:
show_tokens(text, "microsoft/Phi-3-mini-4k-instruct")

 
 English and C AP IT AL IZ ATION 
 � � � �  � � � 
 show _ to kens False None elif == >= else : two tabs :"    " Three tabs : "       " 
 1 2 . 0 * 5 0 = 6 0 0 
 